In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
import mri_data.file_manager as fm
from mri_data import utils
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
#! Set these variables
work_dir_name = "choroid_resegment1"
train_dataset_file_name = "training-dataset-desktop1.json"
prediction_postfix = "choroid_resegment_pred"
task_name = "infer_choroid"
modalities = ["flair", "t1"]
save_dir = Path("/media/smbshare/3Tpioneer_bids_predictions")

In [4]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
drive_root = Path("/media/smbshare")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
inference_root = drive_root / "3Tpioneer_bids_predictions"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
ensemble_out_dir = work_dir / "ensemble_output"


prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_resegment_pred.nii.gz


In [5]:
subjects = [
    1010,
    1011,
    1019,
    1033,
    1065,
    1080,
    1109,
    1119,
    1152,
    1163,
    1188,
    1191,
    1234,
    1259,
    1265,
    1272,
    1280,
    1293,
    1321,
    1355,
    1437,
    1486,
    1498,
    1518,
    1540,
    1547,
    1548,
    2081,
    2083,
    2097,
    2126,
    2132,
    2144,
    2146,
    2164,
    2187,
]

In [6]:
def not_has_subject(scan, subjects: list[int]) -> bool:
    if int(scan.subid) in subjects:
        return False
    else:
        return True

In [9]:
inference_proc = DataSetProcesser.new_dataset(
    inference_root, scan_3Tpioneer_bids, filters=[fm.filter_first_ses],
    label="choroid_resegment1.pineal1.pituitary1.nii.gz"
)
inference_proc.dataset.sort()
inference_proc.filter([not_has_subject], [(subjects,)])
inference_dataset = inference_proc.dataset

350


In [10]:
dataset = DataSet.dataset_like(inference_dataset)
dataset.dataroot = dataroot
dataset.sort()
dataset_proc = DataSetProcesser(dataset)

In [11]:
for scan, inference in zip(dataset, inference_dataset):
    print(scan.subid, inference.subid)

1001 1001
1002 1002
1003 1003
1004 1004
1005 1005
1007 1007
1012 1012
1013 1013
1015 1015
1016 1016
1017 1017
1020 1020
1021 1021
1022 1022
1023 1023
1024 1024
1026 1026
1027 1027
1028 1028
1029 1029
1030 1030
1031 1031
1036 1036
1037 1037
1038 1038
1039 1039
1042 1042
1044 1044
1045 1045
1046 1046
1047 1047
1049 1049
1050 1050
1052 1052
1053 1053
1054 1054
1057 1057
1058 1058
1061 1061
1062 1062
1063 1063
1064 1064
1066 1066
1068 1068
1069 1069
1070 1070
1071 1071
1072 1072
1074 1074
1076 1076
1077 1077
1078 1078
1079 1079
1082 1082
1083 1083
1085 1085
1087 1087
1088 1088
1089 1089
1090 1090
1093 1093
1094 1094
1097 1097
1098 1098
1100 1100
1101 1101
1102 1102
1103 1103
1104 1104
1105 1105
1106 1106
1107 1107
1108 1108
1111 1111
1112 1112
1113 1113
1115 1115
1116 1116
1117 1117
1118 1118
1121 1121
1122 1122
1124 1124
1125 1125
1126 1126
1128 1128
1129 1129
1130 1130
1131 1131
1132 1132
1133 1133
1134 1134
1135 1135
1136 1136
1138 1138
1140 1140
1141 1141
1142 1142
1143 1143
1144 1144


In [12]:
for scan, inference in zip(dataset, inference_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    prediction = inference.label_path
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1001/ses-20170215/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1001/ses-20170215/t1.nii.gz -s /media/smbshare/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/choroid_resegment1.pineal1.pituitary1.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1002/ses-20200521/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1002/ses-20200521/t1.nii.gz -s /media/smbshare/3Tpioneer_bids_predictions/sub-ms1002/ses-20200521/choroid_resegment1.pineal1.pituitary1.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1003/ses-20170329/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1003/ses-20170329/t1.nii.gz -s /media/smbshare/3Tpioneer_bids_predictions/sub-ms1003/ses-20170329/choroid_resegment1.pineal1.pituitary1.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1004/ses-20190906/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1004/ses-20190906/t1.nii.gz -s /media/smbshare/3Tpioneer_bids_predictions/sub-ms1004/ses-201909